In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
# dua vao drive
drive_path = '/content/drive/MyDrive/VIN_NLP/ThanhQuang_NLP/PAWS_Quang_branch'
os.chdir(drive_path)

In [4]:
pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 3.3 MB 35.3 MB/s 
     |████████████████████████████████| 895 kB 50.0 MB/s 
     |████████████████████████████████| 61 kB 492 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
pip install SentencePiece

     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [6]:
# change your index for translate here
#Total length for train file: 49401
# - Hoa: 0 - 25000
# - Tuan: 25000 - 37000
# - Hong Quang: 37000 - 49401
# - Thanh Quang: val and test file 16k rows
START_INDEX = 0
END_INDEX = 25000

In [7]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
from tqdm import tqdm
import csv
import os.path
import pandas as pd

In [12]:
# chay 1 lan de luu vao drive
!mkdir data
!wget https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz -P data
!tar -xvf data/paws_wiki_labeled_final.tar.gz -C data
!mv data/final/* data
!rm -r data/final
!rm -r data/paws_wiki_labeled_final.tar.gz

--2021-12-22 02:35:20--  https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.182.128, 173.194.194.128, 173.194.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.182.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4687157 (4.5M) [application/gzip]
Saving to: ‘data/paws_wiki_labeled_final.tar.gz’

paws_wiki_labeled_f 100%[===================>]   4.47M  --.-KB/s    in 0.1s    

2021-12-22 02:35:21 (35.6 MB/s) - ‘data/paws_wiki_labeled_final.tar.gz’ saved [4687157/4687157]

final/test.tsv
final/
final/train.tsv
final/dev.tsv


In [8]:
df = pd.read_csv('data/dev.tsv',sep = '\t')

In [9]:
if END_INDEX > len(df):
  END_INDEX == len(df)

In [10]:
if torch.cuda.is_available():       
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [11]:
model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-en-vi-base")
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-en-vi-base")
model.to(device)

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (w

In [ ]:
csv_out_file_name = 'paws_trans_devset_' + str(START_INDEX) + '_' + str(END_INDEX) + '.csv'

def load_checkpoint_index():
    if os.path.isfile('checkpoint_index.txt'):
        print ("output file File exist")
    else:
        print ("File not exist, create new data file")
        f = open("checkpoint_index.txt", "w")
        f.write(f'0')
        f.close()
        
    f = open("checkpoint_index.txt", "r")
    index = int(f.read())
    f.close()
    return index



if os.path.isfile(csv_out_file_name):
    print ("output file File exist")
else:
    print ("File not exist, create new data file")
    new_df = pd.DataFrame(columns = ['index','sentence1','sentence2'])
    new_df.to_csv(csv_out_file_name,index = False)



resume_extract_index = load_checkpoint_index()
    
src_path = 'data/dev.tsv'
if resume_extract_index != 0:
    resume_extract_index += 1


df = pd.read_csv(src_path, sep="\t")
df = df[resume_extract_index:END_INDEX]


original_index_list = []
sentence1_list = []
sentence2_list = []
df_output_iter = pd.DataFrame(columns = ['index','sentence1','sentence2'])
for index, row in tqdm(df.iterrows()):
    if row['label'] == 0:
      sentence1 = row['sentence1']
      sentence2 = row['sentence2']

      tokenized_text = tokenizer.encode(sentence1, return_tensors="pt").to(device)
      model.eval()
      summary_ids = model.generate(
                          tokenized_text,
                          max_length=128, 
                          num_beams=5,
                          repetition_penalty=2.5, 
                          length_penalty=1.0, 
                          early_stopping=True
                      )
      output1 = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

      tokenized_text = tokenizer.encode(sentence2, return_tensors="pt").to(device)
      model.eval()
      summary_ids = model.generate(
                          tokenized_text,
                          max_length=128, 
                          num_beams=5,
                          repetition_penalty=2.5, 
                          length_penalty=1.0, 
                          early_stopping=True
                      )
      output2 = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  #     sentence1_list.append(output1)
  #     sentence2_list.append(output2)
      df_output_iter['index'] = row['id']
      df_output_iter['sentence1'] = [str(output1)]
      df_output_iter['sentence2'] = [str(output2)]
      df_output_iter.to_csv(csv_out_file_name, mode='a', header=False, index = False)
      f = open("checkpoint_index.txt", "w")
      f.write(f'{str(index)}')
      f.close()

output file File exist
output file File exist


154it [13:28,  3.41s/it]